In [1]:
import pandas as pd
import numpy as np
import sys
import re
import os
import pandas as pd
sys.path.append("../automaterialsdata/")

from html_table_extractor.extractor import Extractor
from bs4 import BeautifulSoup

from table_extractor import get_all_files_with_extension
from table_extractor import gather_properties_from_full_table


In [2]:
RSC_htmls = get_all_files_with_extension('/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/', 'html')
RSC_htmls = sorted(RSC_htmls, key=lambda x: int(x.split('/')[-1][:-5]))
RSC_htmls[:10]

['/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/14.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/15.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/33.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/34.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/46.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/49.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/56.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/58.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/59.html',
 '/Users/hanghu/Desktop/msedatapipeline/JPCL2018-papers_notPDF/html/RSC/60.html']

In [3]:
searching_patterns = [['VOC', 'VOC(V)'], 
                      ['JSC', 'Jsc(mAcm−2)','Avg.Jsc±std(mAcm−2)'], 
                      ['FF', 'FF(%)'],
                      ['PCE', 'PCE(%)', 'Avg.PCE±std(%)', 'Avg.PCE', 'Max.PCE']]
output_headers = ['V OC (V)', 'J SC (mA/cm-2)', 'FF (%)', 'PCE (%)'] 

all_properties = pd.DataFrame()
paper_count = -1

for f in RSC_htmls:
    
    paper_count += 1

    df = pd.read_html(f, encoding='utf-8')
    headers = [ [table.columns.tolist()] for table in df]
    
    for i in range(len(df)):
        table = df[i]
        header = headers[i]
        
        properties_table = gather_properties_from_full_table(table, header, searching_patterns, output_headers)
        if properties_table is None: continue 
        
#         matched_table = pd.DataFrame()
#         r_headers = reorder_headers(header)

#         for col in range(len(r_headers)):
#             for p, h in zip(searching_patterns, output_headers):
#                 if match_string(p, r_headers[col]):
#                     matched_table[0] = table.iloc[:,0]
#                     matched_table[1] = ''
#                     matched_table[2] = ''
#                     if matched_table[0].isnull().values.any(): 
#                         matched_table[1] = table.iloc[:,1]
#                     if matched_table[1].isnull().values.any(): 
#                         matched_table[2] = table.iloc[:,2]
#                     matched_table[h] = table.iloc[:, col]
        global_idx = []
        for row in range(len(properties_table.index)):
            to_append = str(paper_count ) + '_' + str(i) + '_' + str(row) #Article Index + Table Index + Row Index 
            global_idx.append(to_append)
        properties_table.index = global_idx
        all_properties = pd.concat([all_properties, properties_table], sort=False)

    #all_properties.to_csv('HTML Table Extractor Reuslts.csv',encoding='utf-8')

In [4]:
all_properties.head(5)

,0,1,2,V OC (V),J SC (mA/cm-2),FF (%),PCE (%)
0_5_0,PPDTBT,,,0.76,9.77,0.58,4.27/4.12
0_5_1,PPDTBT,,,0.81,10.40,0.61,5.08/4.89
0_5_2,PPDTFBT,,,0.76,10.6,0.59,4.72/4.38
0_5_3,PPDTFBT,,,0.81,10.2,0.62,5.11/5.02
0_5_4,PPDT2FBT,,,0.78,12.9,0.71,7.18/6.94
